<img src="files/model_1_outline.png">

# change log
### Epoch = 100
### Shuffle=False


sequence generator shape (corrected) (b, n, w, h, c)

In [2]:
import tensorflow as tf
import datetime
import os
import glob
import keras

In [3]:
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [4]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
# from patrice's blogpost
from keras_video import VideoFrameGenerator

In [6]:
classes = ['CalotTriangleDissection',
 'CleaningCoagulation',
 'ClippingCutting',
 'GallbladderDissection',
 'GallbladderPackaging',
 'GallbladderRetraction',
 'Preparation']


classes.sort()
print(classes)

['CalotTriangleDissection', 'CleaningCoagulation', 'ClippingCutting', 'GallbladderDissection', 'GallbladderPackaging', 'GallbladderRetraction', 'Preparation']


In [7]:
# some global params
SIZE = (224, 224) # height and width of frame pxl by pxl
CHANNELS = 3 # RGB or whatever
NBFRAME = 5 # num frames in sequence 
BS = 8 # Batch size

In [8]:
# pattern to get videos and classes
glob_pattern='../../cholec80/model_data_full/{classname}/*.avi'


In [9]:
# for data augmentation
# data_aug = keras.preprocessing.image.ImageDataGenerator(
#     zoom_range=.1,
#     horizontal_flip=True,
#     rotation_range=8,
#     width_shift_range=.2,
#     height_shift_range=.2)

In [10]:
# Create video frame generator
train = VideoFrameGenerator(
    classes=classes, 
    glob_pattern=glob_pattern,
    nb_frames=NBFRAME,
    split_val=.33, 
    shuffle=False,
    batch_size=BS,
    target_shape=SIZE,
    nb_channel=CHANNELS,
    use_frame_cache=True)

class CalotTriangleDissection, validation count: 11743, train count: 23844
class CleaningCoagulation, validation count: 2322, train count: 4717
class ClippingCutting, validation count: 2191, train count: 4450
class GallbladderDissection, validation count: 8319, train count: 16892
class GallbladderPackaging, validation count: 1237, train count: 2514
class GallbladderRetraction, validation count: 1074, train count: 2182
class Preparation, validation count: 1397, train count: 2838
Total data: 7 classes for 57437 files for train


In [11]:
# getting validation data
valid = train.get_validation_generator()

Total data: 7 classes for 28283 files for validation


In [12]:
import keras_video.utils
#keras_video.utils.show_sample(train)

In [18]:
#valid.files
from keras import Model

## BUILD CONV NET

In [19]:
from keras.layers import Conv2D, BatchNormalization, \
    MaxPool2D, GlobalMaxPool2D
#from keras.applications.mobilenet import MobileNet
from keras.applications import ResNet50 
def build_mobilenet(shape=(224, 224, 3), nbout=3):
#     model = MobileNet(
#         include_top=False,
#         input_shape=shape,
#         weights='imagenet')
#     # Keep 9 layers to train﻿﻿
#     trainable = 9
#     for layer in model.layers[:-trainable]:
#         layer.trainable = False
#     for layer in model.layers[-trainable:]:
#         layer.trainable = True
    model = ResNet50(include_top=True, weights='imagenet')
    #return keras.Sequential([model])
    return Model(inputs=model.input, outputs=model.layers[-2].output)

## Build LSTM

In [20]:
from keras.layers import TimeDistributed, GRU, Dense, Dropout, LSTM
from keras.initializers import GlorotNormal
# Shape (5, 112, 112, 3) 5 - time sequence length 112x112 = height vs width 3 - num channels
def action_model(shape=(5, 112, 112, 3), nbout=3):
    # Create our convnet with (112, 112, 3) input shape
    convnet = build_mobilenet(shape[1:])
    
    # then create our final model
    model = keras.Sequential()
    # add the convnet with (5, 112, 112, 3) shape
    # KEY = allows you to add a time sequence to a layer one at a time
    model.add(TimeDistributed(convnet, input_shape=shape))
    model.add(Dense(512))
    # here, you can also use GRU or LSTM
    #model.add(LSTM(512, kernel_initializer='glorot_normal', return_sequences=False, input_shape=(2048, 5)))
    #model.add(LSTM(512, kernel_initializer='glorot_normal', return_sequences=False,  input_shape=(2048, 5, 8)))
    model.add(LSTM(512, kernel_initializer='glorot_normal'))
    # and finally, we make a decision network
    model.add(Dense(nbout, activation='linear'))
    return model

## this is where you tell the model how to train - loss function, weight update mechanism

In [21]:
INSHAPE=(NBFRAME,) + SIZE + (CHANNELS,) # (5, 128, 128, 3)
print(INSHAPE)
# action model - GRU set up for Time shifted CNN
model = action_model(INSHAPE, len(classes))

#this is where you tell the model how to train - loss function, weight update mechanism
optimizer = keras.optimizers.Adam(0.001)
model.compile(
    optimizer,
    'categorical_crossentropy',
    metrics=['acc']
)

(5, 224, 224, 3)


## epochs, call backs

In [22]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

print('logs for this run are here: {}'.format(log_dir))

logs for this run are here: logs/fit/20210621-231113


In [23]:
EPOCHS=100
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
# create a "chkp" directory before to run that
# because ModelCheckpoint will write models inside
callbacks = [
    keras.callbacks.ReduceLROnPlateau(verbose=1),
    keras.callbacks.ModelCheckpoint(
        'model2_3_1_chkp/weights.{epoch:02d}-{val_loss:.2f}.hdf5',
        verbose=1),
    tensorboard_callback
]

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
model.fit(
    train,
    validation_data=valid,
    verbose=1,
    epochs=EPOCHS,
    callbacks=callbacks
)

Epoch 1/100
7179/7179 [==============================] - ETA: 0s - loss: 10.5344 - acc: 0.2966

Frame count is not OK for video ../../cholec80/model_data_full/GallbladderDissection\GallbladderDissection24977.avi, 0 total, 0 extracted


ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted:  MemoryError: Unable to allocate 23.0 MiB for an array with shape (8, 5, 224, 224, 3) and data type float32
Traceback (most recent call last):

  File "C:\Users\calvinap\miniconda3\envs\hopethisworks3\lib\site-packages\tensorflow\python\ops\script_ops.py", line 249, in __call__
    ret = func(*args)

  File "C:\Users\calvinap\miniconda3\envs\hopethisworks3\lib\site-packages\tensorflow\python\autograph\impl\api.py", line 620, in wrapper
    return func(*args, **kwargs)

  File "C:\Users\calvinap\miniconda3\envs\hopethisworks3\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py", line 891, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "C:\Users\calvinap\miniconda3\envs\hopethisworks3\lib\site-packages\tensorflow\python\keras\engine\data_adapter.py", line 807, in wrapped_generator
    for data in generator_fn():

  File "C:\Users\calvinap\miniconda3\envs\hopethiswor